In [1]:
from openai import OpenAI



client:OpenAI = OpenAI()

In [ ]:
def travel_map():
    print

In [2]:
# creating assistant

assistant = client.beta.assistants.create(
    name="Travel Assistant",
    instructions="""
        you are a experienced travel assiatant, you suggest the user best places at location which is provided by the user
        you will be suggesting user by calling a custom map function 'travel_map'
        
    """
    model="gpt-3.5-turbo-1106"
    tools=[
        {
            "type":'function',
            "function":{
                "name":"travel_map",

            }
        }
    ]
)